In [ ]:
import time
import os
from shutil import copyfile
import logging
from hbp_nrp_virtual_coach.virtual_coach import VirtualCoach
vc = VirtualCoach(environment='local')

## A transfer function to display the trial number in the frontend

In [ ]:
display_trial_tf = """@nrp.Robot2Neuron()
def display_trial_number(t):
    clientLogger.advertise('%s')
"""

## Define the number of runs and how long each simulation should run for

In [ ]:
simulation_time = 10 # seconds
src = os.environ['HBP'] + '/Experiments/cdp1_mouse/nnStructures/generatedStructures/'
des = os.environ['HBP'] + '/Experiments/cdp1_mouse/nnStructures/'
# list of all files in the generatedStructures directory
nn_structures = os.listdir(src)
#runs = len(nn_structures) # number of trials

In [ ]:
class cdp1Experiment(object):
    
    def __init__(self, runs, simulation_time):
        self.last_status = [None]
        self.runs = nn_structures
        self.simulation_time = simulation_time
        self.sled_position = []

        # disable global logging from the virtual coach
        logging.disable(logging.INFO)
        logging.getLogger('rospy').propagate = False
        logging.getLogger('rosout').propagate = False

    def wait_condition(self, timeout, condition):
        start = time.time()
        while time.time() < start + timeout:
            time.sleep(0.25)
            if condition(self.last_status[0]):
                return
        raise Exception('Condition check failed')        

    def on_status(self, status):
        self.last_status[0] = status

    def run_experiment(self):
        for i in range(len(self.runs)):
            # copy neural network structure file
            copyfile(src + self.runs[i], des + 'closedLoopMouse.txt')

            # launch experiment
            try:
                self.sim = vc.launch_experiment('ExDMouseCDP1')
            except:
                time.sleep(1)
            time.sleep(20)
            
            self.sim.register_status_callback(self.on_status)
            self.wait_condition(10, lambda x: x is not None)
            self.sim.add_transfer_function(display_trial_tf % "Trial {}".format(i + 1))

            # start the experiment
            self.sim.start()
            # wait until the desired simulation time is reached
            self.wait_condition(1000, lambda x: x['simulationTime'] == self.simulation_time)
            self.sim.pause()
            # save the sled positions to compare later 
            self.sled_position.append(self.sim.get_csv_data('all_joints_positions.csv'))
            self.sim.stop()
            time.sleep(20)

## Run the experiment

In [ ]:
cdp1_experiment = cdp1Experiment(nn_structures, simulation_time)
cdp1_experiment.run_experiment()

## Plot the sled position for each run

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from cycler import cycler
width = 8
height = 8
plt.figure(figsize=(width, height))
for j in range(len(cdp1_experiment.sled_position)):
    sled = [item[1:] for item in cdp1_experiment.sled_position[j] if item[0] == 'cdp1_msled::world_sled']
    plt.rc('axes', prop_cycle=(cycler('color', ['#75bbfd', '#029386', '#9a0eea', '#f10c45', '#cdc50a', '#343837', '#7ebd01', '#3c0008', '#c8aca9', '#030764'])))
    plt.xlabel('Time [s]')
    #plt.xlim(0, 10)
    plt.ylabel('Sled position [m]')
    plt.legend(loc=1)
    plt.plot([float(i[0]) for i in sled], [float(i[1]) for i in sled])